In [1354]:
import pandas as pd
import time, random
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import sys

In [1355]:
n_samples=300
n_components=3
k=3

In [1356]:
X, y_true = make_blobs(
    n_samples=n_samples, centers=n_components, n_features=3, cluster_std=1.0, random_state=0
)
X = X[:, ::-1]

In [1357]:
def min_max(column, min, max):
    X_std = (column - np.min(column)) / (np.max(column) - np.min(column))
    return X_std * (max - min) + min

In [1358]:
dataset = pd.DataFrame(data=X, columns=["avaliacao_1", "avaliacao_2", "idade"])
dataset['avaliacao_1'] = min_max(dataset['avaliacao_1'], min=0, max=10)
dataset['avaliacao_2'] = min_max(dataset['avaliacao_2'], min=0, max=10)
dataset['idade'] = [int(x) for x in min_max(dataset['idade'], min=18, max=60)]
dataset.to_csv("dataset.csv", sep=",", index=False)

In [1359]:
# caculate euclidean distance between two vectors
def calculate_distance(arr1, arr2):
    return np.linalg.norm(np.asarray(arr1) - np.asarray(arr2), ord=None)

def kmeans(dataset, k, iteration):
    row_dataset = dataset.shape[0]
    clusters = [0] * row_dataset
    # choosing initial seeds
    centroids = pd.DataFrame(data=np.random.random_sample((k, dataset.shape[1])), columns=sorted(dataset.columns))
    # id_samples = np.random.choice(row_dataset, k, replace=False)
    # centroids = dataset.loc[id_samples] # * random.uniform(1.0, 9.0)
    # centroids.reset_index(inplace=True)
    # centroids.drop(columns=["index"], inplace=True)
    cumulative_centroids = centroids.copy()
    old_sum = [0] * k
    new_sum = [-1] * k
    count_members = np.array([0] * k)
    count = 0
    # kmeans iteration
    iterations_centroids = []
    list_clusters = []
    # iterations_centroids.append(pd.DataFrame(data=st.inverse_transform(centroids.copy()), columns=centroids.columns))
    # iterations_centroids.append(centroids.copy())
    while count < iteration and old_sum != new_sum:
        new_sum = old_sum
        old_sum = [0] * k
        for j in range(0, row_dataset):
           min_distance = sys.float_info.max
           cluster_id = 0
           for i in range(0, k): 
               distance = calculate_distance(dataset.loc[j], centroids.loc[i])
               if min_distance > distance:
                   min_distance = distance
                   cluster_id = i
           old_sum[cluster_id] = old_sum[cluster_id] + min_distance
           clusters[j] = cluster_id
           cumulative_centroids.loc[cluster_id] = cumulative_centroids.loc[cluster_id] + dataset.loc[j]
           count_members[cluster_id] =  count_members[cluster_id] + 1 
        iterations_centroids.append(centroids.copy())
        list_clusters.append(clusters)
        # updating cetroids
        for i in range(0, k):
           if count_members[i] != 0:
               centroids.loc[i] = cumulative_centroids.loc[i] / count_members[i]  
           else:
               # selecting new seed
               # centroids.loc[i] = dataset.loc[np.random.choice(row_dataset, 1, replace=False)]
               print("cai aqui")
               pass
        # iterations_centroids.append(pd.DataFrame(data=st.inverse_transform(centroids.copy()), columns=centroids.columns)) 
        count_members = [0] * k  
        clusters = [0] * row_dataset
        cumulative_centroids = cumulative_centroids * 0.0
        count = count + 1
        # return cluster labels and final centroids
    return [clusters, centroids, iterations_centroids, len(iterations_centroids), list_clusters]

# normalize dataset
st = StandardScaler()
st.fit(dataset)
dataset = pd.DataFrame(data=st.transform(dataset), columns=dataset.columns)
model = kmeans(dataset=dataset[['avaliacao_1', 'avaliacao_2']], k=k, iteration=1000)
iterations = model[2]
model[3]


6

In [1360]:
clusters = model[4]
print(model[3])
print(len(clusters))

6
6


In [1361]:
x = 'avaliacao_1'
y = 'avaliacao_2'

In [1362]:
plt.figure(figsize=(10, 10))
plt.scatter(x=x, s=300, y=y, edgecolors='black', c='#B0C4DE', data=dataset)
plt.xlabel("Idade dos alunos", fontsize=18)
plt.title("Afinidade em Programação I", fontsize=24)
plt.ylabel("Score de Afinidade - Programação I", fontsize=18)
plt.savefig(f"init.png")
plt.figure().clear()
plt.close()
plt.cla()
plt.clf()

<Figure size 720x720 with 0 Axes>

In [1363]:
plt.figure(figsize=(10, 10))
plt.scatter(x=x, s=300, y=y, edgecolors='black', c='#B0C4DE', data=dataset)
plt.scatter(x=x, y=y, edgecolors='red', c=iterations[0].index, s=3000, marker='*', data=iterations[0])
plt.xlabel("Idade dos alunos", fontsize=18)
plt.title("Afinidade em Programação I", fontsize=24)
plt.text(-1.5, 1.3, "1 - Escolhe centros iniciais", fontsize=16, style='italic',
        bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10})
plt.ylabel("Score de Afinidade - Programação I", fontsize=18)
plt.savefig(f"iter_{0}.png")
plt.figure().clear()
plt.close()
plt.cla()
plt.clf()

<Figure size 720x720 with 0 Axes>

In [1364]:
plt.figure(figsize=(10, 10))
plt.scatter(x=x, s=300, y=y, edgecolors='black', c=clusters[0], data=dataset)
plt.scatter(x=x, y=y, edgecolors='red', c=iterations[0].index, s=3000, marker='*', data=iterations[0])
plt.xlabel("Idade dos alunos", fontsize=18)
plt.title("Afinidade em Programação I", fontsize=24)
plt.text(-1.5, 1.3, "2 - Atribui cada obj. ao centro + prox.", fontsize=16, style='italic',
        bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10})
plt.ylabel("Score de Afinidade - Programação I", fontsize=18)
plt.savefig("iter_atri.png")
plt.figure().clear()
plt.close()
plt.cla()
plt.clf()

<Figure size 720x720 with 0 Axes>

In [1365]:
plt.figure(figsize=(10, 10))
plt.scatter(x=x, s=300, y=y, edgecolors='black', c=clusters[0], data=dataset)
plt.scatter(x=x, y=y, edgecolors='red', c=list(set(clusters[0])), s=3000, marker='*', data=iterations[1])
plt.xlabel("Idade dos alunos", fontsize=18)
plt.title("Afinidade em Programação I", fontsize=24)
plt.text(-1.5, 1.3, "3 - Move cada centro para o meio do grupo", fontsize=16, style='italic',
        bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10})
plt.ylabel("Score de Afinidade - Programação I", fontsize=18)
plt.savefig("iter_move.png")
plt.figure().clear()
plt.close()
plt.cla()
plt.clf()

<Figure size 720x720 with 0 Axes>

In [1366]:
for _iter in range(1, model[3]):
    plt.figure(figsize=(10, 10))
    plt.scatter(x=x, s=300, y=y, edgecolors='black', c=clusters[_iter], data=dataset)
    plt.scatter(x=x, y=y, edgecolors='red', c=list(set(clusters[_iter])), s=3000, marker='*', data=iterations[_iter])
    plt.text(-1.5, 1.3, "4 - Repete os passos 1,2 e 3", fontsize=16, style='italic',
        bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10})
    plt.xlabel("Idade dos alunos", fontsize=18)
    plt.title("Afinidade em Programação I", fontsize=24)
    plt.ylabel("Score de Afinidade - Programação I", fontsize=18)
    plt.savefig(f"iter_{_iter}.png")
    plt.figure().clear()
    plt.close()
    plt.cla()
    plt.clf()

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>